In [1]:
import pandas as pd
import requests
from io import StringIO

In [2]:
access_token = "0baaf4e1832be5782d9833889675dccf"
headers = {"Authorization": f"Bearer {access_token}"}

In [3]:
base_url = "https://it-hiring.blackbird.vc"
crm_url = f"{base_url}/api/data/crm"
form_url = f"{base_url}/api/data/form-submissions"

In [4]:
crm_response = requests.get(crm_url, headers=headers)
crm_df = pd.read_csv(StringIO(crm_response.text))
print("CRM data loaded.")
display(crm_df.head())

CRM data loaded.


,id,first,last,email,phone,last contact date,last contact text,all contact text
0,bfx-1116,Jack,North,jn231@hotmail.test.com,6.139008e+10,2025-01-19,Gave positive feedback about the overall organ...,2025-01-19 - Gave positive feedback about the ...
1,bfx-9131,Alice,South,asouth@bigpond.com,NaN,2023-07-22,Found the latest newsletter about 'Q4 2024 Com...,2023-07-22 - Found the latest newsletter about...
2,bfx-3367,Kanye,West,big.dawg.99@hotmail.test.com,6.137481e+10,2022-07-04,Engaged actively during the Q&A session at the...,2022-07-04 - Engaged actively during the Q&A s...
3,bfx-7190,Bev,East,NaN,6.137492e+10,2023-11-16,Gave positive feedback about the overall organ...,2023-11-16 - Gave positive feedback about the ...
4,bfx-9147,Margaret,Piper,mpiper@black-bird-email.com,NaN,2023-02-10,Really enjoyed the networking opportunities at...,2023-02-10 - Really enjoyed the networking opp...


In [5]:
form_response = requests.get(form_url, headers=headers)
form_data = form_response.json()
form_df = pd.json_normalize(form_data["data"])
print("Feedback form data loaded.")
display(form_df.head())

Feedback form data loaded.


,email,message,name,phone,timestamp
0,jn231@hotmail.test.com,The offsite was fantastic,Jack North,+61390084499,2025-04-01
1,asouth@bigpond.com,More coffee next time,Alison South,+61383128784,2025-04-01
2,kanye@kanye.com,More tunes,kanye,+61374813443,2025-04-01
3,beverly.east@yahoo.com,Location could have been better,Beverly East,+61374921338,2025-04-02
4,mpiper@black-bird-email.com,I had some great conversations,Margeret piper,+61364574723,2025-04-02


In [ ]:
form_df[["first_form", "last_form"]] = form_df["name"].str.extract(r'^(\S+)\s+(.*)$')

form_df.rename(columns={
    "email": "email",
    "phone": "phone_form",
    "timestamp": "last contact date",
    "message": "last contact text"
}, inplace=True)

In [8]:
form_df["all contact text"] = form_df["last contact text"]

In [9]:
merged_df = pd.merge(crm_df, form_df, on="email", how="outer", suffixes=("", "_form"))


In [10]:
merged_df["first"] = merged_df["first"].combine_first(merged_df["first_form"])
merged_df["last"] = merged_df["last"].combine_first(merged_df["last_form"])
merged_df["phone"] = merged_df["phone"].combine_first(merged_df["phone_form"])


In [11]:
merged_df["last contact date"] = merged_df["last contact date_form"]
merged_df["last contact text"] = merged_df["last contact text_form"]


In [12]:
merged_df["all contact text"] = merged_df["all contact text"].fillna("") + " | " + merged_df["last contact text"].fillna("")


In [13]:
if merged_df["id"].isnull().any():
    max_id = pd.to_numeric(merged_df["id"].str.extract(r"(\d+)", expand=False), errors="coerce").dropna().astype(int).max()
    new_ids = [f"bfx-{max_id + i + 1}" for i in range(merged_df["id"].isnull().sum())]
    merged_df.loc[merged_df["id"].isnull(), "id"] = new_ids


In [14]:
final_df = merged_df[[
    "id", "first", "last", "email", "phone",
    "last contact date", "last contact text", "all contact text"
]]

In [15]:
final_df.to_csv("crm_update.csv", index=False)
print("crm_update.csv has been generated.")

crm_update.csv has been generated.
